In [1]:
import pandas as pd
import numpy as np
import requests

pd.set_option('display.max_rows', 100)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge

import warnings
warnings.filterwarnings("ignore")

import pickle

import os

In [2]:
stocks = {
    'fwry' : 'https://www.investing.com/equities/fawry-banking-and-payment-historical-data',
    'abuk' : 'https://www.investing.com/equities/abou-kir-fertilizers-historical-data',
    'alcn' : 'https://www.investing.com/equities/alexandria-containers-and-goods-historical-data',
    'ekhoa' : 'https://www.investing.com/equities/egypt-kuwait-holding-historical-data',
    'mfpc': 'https://www.investing.com/equities/misr-fertilizers-production-co-sae-historical-data',
    'atlc': 'https://www.investing.com/equities/al-tawfeek-financial-historical-data',
    'auto': 'https://www.investing.com/equities/gb-auto-s.a.e-historical-data',
    'efid': 'https://www.investing.com/equities/edita-food-industries-historical-data',
    'orhd': 'https://www.investing.com/equities/orascom-hotels-and-development-historical-data',
    'rmda': 'https://www.investing.com/equities/tenth-of-ramadan-for-pharmaceutical-historical-data',
    'comi': 'https://www.investing.com/equities/com-intl-bk-historical-data',
    'etel': 'https://www.investing.com/equities/telecom-egypt-historical-data',
    'olfi': 'https://www.investing.com/equities/obour-land-for-food-industries-historical-data',
    'qnba': 'https://www.investing.com/equities/natl-soc-gen-b-historical-data',
    'adib': 'https://www.investing.com/equities/abu-dhabi-islamic-bank-egypt-historical-data',
    'amoc': 'https://www.investing.com/equities/alx-mineral-oi-historical-data',
    'egch' : 'https://www.investing.com/equities/egypt-chem-ind-historical-data',
    'csag' : 'https://www.investing.com/equities/canal-shipping-historical-data',
    'emfd' : 'https://www.investing.com/equities/emaar-misr-for-development-sae-historical-data',
    'orwe' : 'https://www.investing.com/equities/oriental-weave-historical-data',
    'ORAS' : 'https://www.investing.com/equities/orascom-construction-ltd-historical-data',
    'RACC' : 'https://www.investing.com/equities/raya-contact-center-historical-data',
    'skpc' : 'https://www.investing.com/equities/sidi-kerir-pet-historical-data',
    'swdy' : 'https://www.investing.com/equities/elsewedy-cable-historical-data',
    'east' : 'https://www.investing.com/equities/eastern-co-historical-data',
    'cira' : 'https://www.investing.com/equities/cairo-investment---re-development-historical-data',
    'heli' : 'https://www.investing.com/equities/heliopolis-housing-historical-data',
    'mtie' : 'https://www.investing.com/equities/mm-group-for-industry-historical-data',
    'saud' : 'https://www.investing.com/equities/barka-egypt-ba-historical-data',
    'hdbk' : 'https://www.investing.com/equities/housing---dev-historical-data',
    'tmgh' : 'https://www.investing.com/equities/t-m-g-holding-historical-data',
    'arcc' : 'https://www.investing.com/equities/arabian-cement-co-sae-historical-data',
    'ccap' : 'https://www.investing.com/equities/citadel-capita-historical-data',
    'cich' : 'https://www.investing.com/equities/ci-capital-historical-data',
    'clho' : 'https://www.investing.com/equities/cleopatra-hospital-historical-data',
}

In [3]:
tables = {}
for stock, link in stocks.items():
    url = requests.get(link)
    try:
        data = pd.read_html(url.text)[1]
        data.columns = ['Date','Price','Open','Max','Min','Vol','Change']
        tables[stock] = data
    except:
        pass

In [4]:
def predict(table):
    table_ = table.copy()

    table_[['next_Max','next_Min']] = table_[['Max','Min']][1:].reset_index()[['Max','Min']]
    # table_['DayOfWeek'] = pd.to_datetime(table_.Date).dt.day_of_week

    # table_.Change = table_.Change.str[:-1]
    # table_.Change = table_.Change.astype(float)

    # table_.Vol[table_.Vol.str[-1] == 'M'] = table_.Vol.str[:-1].astype(float) * 10**6
    # table_.Vol[table_.Vol.str[-1] == 'K'] = table_.Vol.str[:-1].astype(float) * 10**3
    # table_.Vol = table_.Vol.astype(float)

    # Creating train and test sets:
    min_ = table_.next_Min[:-1]
    max_ = table_.next_Max[:-1]
    train = table_.drop(['Date','next_Min','next_Max','Change','Vol'],axis=1)

    # Scaling the data. 
    data_scaler = StandardScaler()
    max_scaler = StandardScaler()
    min_scaler = StandardScaler()

    scaled_data = data_scaler.fit_transform(train.values)
    scaled_min = min_scaler.fit_transform(min_.values.reshape(-1, 1))
    scaled_max = max_scaler.fit_transform(max_.values.reshape(-1, 1))

    scaled_data= pd.DataFrame(scaled_data, index=train.index, columns=train.columns)

    Xt = scaled_data.iloc[:-1,:]
    Xv = scaled_data.iloc[-1,:].values.reshape(1, -1)
    
    
    KNN_grid = {'n_neighbors' : list(range(3,10,2))}

    knn = KNeighborsRegressor()

    grid_min = GridSearchCV(knn, KNN_grid).fit(Xt,scaled_min)
    grid_max = GridSearchCV(knn, KNN_grid).fit(Xt,scaled_max)

    knnmin = grid_min.best_estimator_
    knnmax = grid_max.best_estimator_

    knnmin.fit(Xt,scaled_min)
    minpred = knnmin.predict(Xv)
    minpred = min_scaler.inverse_transform(minpred)

    knnmax.fit(Xt,scaled_max)
    maxpred = knnmax.predict(Xv)
    maxpred = max_scaler.inverse_transform(maxpred)

    return round(minpred[0][0],2),round(maxpred[0][0],2)

In [35]:
names = []
p25 = []
p50 = []
p75 = []
min_ = []
avg = []
max_ = []
pro = []
now = []
should = []
pred_min = []
pred_max = []
is_ = []
rate = []
will = []
q = []

prom = []
is_m = []
ratem = []
p25m = []
p50m = []
p75m = []
mq = []

for stock, table_ in tables.items():

    table = table_.iloc[:7,:].copy()
    prices = pd.concat([table.Price,table.Open,table.Max,table.Min])
    fq = np.quantile(prices,0.25)
    sq = np.quantile(prices,0.5)
    tq = np.quantile(prices,0.75)
    names.append(stock.upper())
    p25.append(fq)
    p50.append(sq)
    p75.append(tq)
    min_.append(table.Min.min())
    avg.append(round(table.Price.mean(),2))
    max_.append(table.Max.max())
    pro.append(round(tq/fq-1,2))
    now.append(table.Price[0])
    predmin,predmax = predict(table_)
    pred_min.append(predmin)
    pred_max.append(predmax)
    if table.Price[0] < predmin:
        will.append('Go Up')
    elif table.Price[0] > predmax:
        will.append('Go Down')
    else:
        will.append('Stand Still')
    go = (table.Price+table.Open+table.Max+table.Min)/4
    go = go.iloc[::-1].reset_index()[0]
    for i in range(len(go)-1):
        go[i] = go[i+1] - go[i]
    go = go[:-1].mean()
    rate.append(round(go,4))
    if go > 0.1:
        is_.append('Going Up')
    elif go < -0.1:
        is_.append('Going Down')
    else:
        is_.append('Static')
    prices = prices.sort_values().to_frame()
    prices['DecileRank'] = [n*100/len(prices) for n in range(0,len(prices))]
    q.append(round(prices[prices[0] == table.Price[0]]['DecileRank'].mean(),2))

    prices_ = pd.concat([table_.Price,table_.Open,table_.Max,table_.Min])
    fq_ = np.quantile(prices_,0.25)
    sq_ = np.quantile(prices_,0.5)
    tq_ = np.quantile(prices_,0.75)
    p25m.append(fq_)
    p50m.append(sq_)
    p75m.append(tq_)
    prom.append(round(tq_/fq_-1,2))
    go_ = (table_.Price+table_.Open+table_.Max+table_.Min)/4
    go_ = go_.iloc[::-1].reset_index()[0]
    for i in range(len(go_)-1):
        go_[i] = go_[i+1] - go_[i]
    go_ = go_[:-1].mean()
    ratem.append(round(go_,4))
    if go_ > 0.05:
        is_m.append('Going Up')
    elif go_ < -0.05:
        is_m.append('Going Down')
    else:
        is_m.append('Static')
    prices_ = prices_.sort_values().to_frame()
    prices_['DecileRank'] = [n*100/len(prices_) for n in range(0,len(prices_))]
    mq.append(round(prices_[prices_[0] == table.Price[0]]['DecileRank'].mean(),2))
    
    if ((table.Price[0] <= fq and round(tq/fq-1,2) >= 0.06)) and not ((go < -0.1)):
        should.append('Buy FW')
    elif (table.Price[0] <= sq and round(tq/fq-1,2) >= 0.12) and not ((go < -0.1)):
        should.append('Buy SW')
    elif (table.Price[0] <= fq_ and round(tq_/fq_-1,2) >= 0.1) and not ((go_ < -0.05)):
        should.append('Buy FM')
    elif ( (table.Price[0] <= sq_ and round(tq_/fq_-1,2) >= 0.2)) and not ((go_ < -0.05)):
        should.append('Buy SM')
    elif (table.Price[0] >= tq_) and not ((go_ > 0.05) or (go > 0.1)):
        should.append('Sell M')
    elif (table.Price[0] >= tq) and not ((go_ > 0.05) or (go > 0.1)):
        should.append('Sell W')
    else:
        should.append('Hold')

    

final_dict = {
    'Stocks' : names,
    'Present Price' : now,
    'Weekly Rate' : q,
    '25%' : p25,
    '50%' : p50,
    '75%' : p75,
    'Min' : min_,
    'Max' : max_,
    'Average' : avg,
    'It Is (Weekly)' : is_,
    'Weekly Going rate' : rate,
    'Weekly Risk Percentage' : pro,
    'It Is (Monthly)' : is_m,
    'Monthly Going rate' : ratem,
    'Monthly Risk Percentage' : prom,
    'Monthly Rate' : mq,
    'Month FQ' : p25m,
    'Month SQ' : p50m,
    'Month TQ' : p75m,
    'You Should' : should,
    'Next Predicted Min' : pred_min,
    'Next Predicted Max' : pred_max,
    'It May' : will
}

Date = table.Date[0].replace('/','-')

cmh = sns.color_palette("Greens", as_cmap=True)
cmg = sns.color_palette("RdYlGn", as_cmap=True)
cmgr = sns.color_palette("RdYlGn_r", as_cmap=True)

fdf = pd.DataFrame(final_dict).sort_values(by=['You Should','Weekly Risk Percentage','Weekly Going rate'],ascending=[True,False,False]).style.background_gradient(cmap=cmh,subset=['Weekly Risk Percentage','Monthly Risk Percentage'])
fdf = fdf.background_gradient(cmap=cmg,subset=['Weekly Going rate','Monthly Going rate'])
fdf = fdf.background_gradient(cmap=cmgr,subset=['Weekly Rate','Monthly Rate'])

fdf.to_excel(Date+'.xlsx',index=False)

fdf.to_excel(f"C:\\Users\LAPTOP WORLD\OneDrive\{Date}.xlsx",index=False)

In [29]:
prices

,0,DecileRank
4,5.04,0.000000
5,5.05,3.571429
3,5.15,7.142857
3,5.27,10.714286
4,5.29,14.285714
4,5.30,17.857143
5,5.31,21.428571
4,5.38,25.000000
1,5.40,28.571429
0,5.41,32.142857
